In [170]:
import finnhub
import pandas as pd
from datetime import datetime, timedelta
import requests
from bs4 import BeautifulSoup
import re
import sqlite3
import time
from sqlite3 import OperationalError

In [6]:
import alpaca_trade_api as api

API_KEY = 'PKQA6MM20L2CQZKAOAHW'
API_SECRET = '4AJeSo8hcSqHa2TNwJv9IS8efb5PNDxHUju3UJjH'
BASE_URL = "https://paper-api.alpaca.markets"

alpaca = api.REST(API_KEY, API_SECRET, BASE_URL)

In [180]:
class trade_idea:
    def __init__(self, symbol, time_frame, position, time_posted, user):
        self.symbol = symbol
        self.time_frame = time_frame
        self.position = position
        self.time_posted = time_posted
        self.user = user
        
class user_rating:
    def __init__(self, username, profit_percentage, num_trades):
        self.username = username
        self.profit_percentage = profit_percentage
        self.num_trades = num_trades
        
def get_trade_ideas_db():    
    con = sqlite3.connect('TradingBotDatabase.db')
    cur = con.cursor()
    trade_ideas = []
    for row in cur.execute(f"SELECT * FROM trade_ideas;"):
        trade_ideas.append(trade_idea(row[0], row[1], row[2], row[3], row[4]))
    return trade_ideas

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn

def get_trade_ideas(begin_page, end_page):
    trade_ideas = []
    for i in range(begin_page, end_page):
        print(i)
        trade_ideas.extend(store_all_data("https://www.tradingview.com/markets/stocks-usa/ideas/page-%d/" % i))
    print(len(trade_ideas))
    return trade_ideas

def store_all_data(URL):
    page = requests.get(URL)
    soup = BeautifulSoup(page.content, "html.parser")
    data = soup.find_all("div", class_="tv-widget-idea js-userlink-popup-anchor")
    trade_ideas = []
    print(len(data))
    for i in data:
        symbol = i.find('a', class_="tv-widget-idea__symbol apply-overflow-tooltip")
        time_frame = i.find_all('span', class_="tv-widget-idea__timeframe")[1]
        position = i.find('span', class_="content-yHuWj4ze badge-idea-content-Naj79Gc7")
        time_posted = i.find('span', class_="tv-card-stats__time")
        user = i.find('span', class_="tv-card-user-info__name")
        if(symbol != None and time_frame != None and position != None and time_posted != None and user != None):
            trade_ideas.append(trade_idea(symbol.text, time_frame.text, position.text, time_posted['data-timestamp'], user.text))          
    return trade_ideas

def calculate_profit(stock, profit_exit, loss_exit):
    buy = 100
    initial_price = stock['open'][0]
    exit1 = profit_exit
    exit2 = loss_exit
    for i in stock['open']:
        if i / initial_price > exit1:
            print("initial_price: ", initial_price, "current price: ", i)
            return (buy * (i / initial_price)) - buy
        if i / initial_price < exit2:
            print("initial_price: ", initial_price, "current price: ", i)
            return (buy * (i / initial_price)) - buy
    return 0

def calculate_total_profit_good_users(trade_ideas, profit_exit_margin, exit_trade_margin, good_users):
    sum = 0
    users = {}
    for i in trade_ideas:
        if i.user not in good_users:
            continue
        try:
            stock = get_stock_db(i.symbol, i.time_posted, time.time() - 86400)
        except OperationalError:
            stock = get_stock_alpaca(i.symbol, i.time_posted, time.time() - 86400)
        if len(stock) < 100:
            continue
        if i.position == "Long":
            profit = calculate_profit(stock, 1+profit_exit_margin, 1-exit_trade_margin)
            sum += profit
            if i.user not in users:
                users[i.user] = [0,0]
            users[i.user][0] += profit
            users[i.user][1] += 1
            print(i.symbol, i.time_frame, i.position, profit)
            
        elif i.position == "Short":
            profit = -calculate_profit(stock, 1+exit_trade_margin, 1-profit_exit_margin)
            sum += profit
            if i.user not in users:
                users[i.user] = [0,0]
            users[i.user][0] += profit
            users[i.user][1] += 1
            print(i.symbol, i.time_frame, i.position, profit)
    print("TOTAL PROFIT: ", sum)
    return users

def calculate_total_profit(trade_ideas, profit_exit_margin, exit_trade_margin):
    sum = 0
    users = {}
    for i in trade_ideas:
        try:
            stock = get_stock_db(i.symbol, i.time_posted, time.time() - 86400)
        except OperationalError:
            stock = get_stock_alpaca(i.symbol, i.time_posted, time.time() - 86400)
        if len(stock) < 100:
            continue
        if i.position == "Long":
            profit = calculate_profit(stock, 1+profit_exit_margin, 1-exit_trade_margin)
            sum += profit
            if i.user not in users:
                users[i.user] = [0,0]
            users[i.user][0] += profit
            users[i.user][1] += 1
            print(i.symbol, i.time_frame, i.position, profit)
            
        elif i.position == "Short":
            profit = -calculate_profit(stock, 1+exit_trade_margin, 1-profit_exit_margin)
            sum += profit
            if i.user not in users:
                users[i.user] = [0,0]
            users[i.user][0] += profit
            users[i.user][1] += 1
            print(i.symbol, i.time_frame, i.position, profit)
    print("TOTAL PROFIT: ", sum)
    return users
    
def get_stock_db(stock_symbol, date_begin, date_end):
    con = sqlite3.connect('TradingBotDatabase.db')
    cur = con.cursor()
    date_begin = datetime.fromtimestamp(int(float(date_begin))).strftime("%Y-%m-%d %H:%M:%S")
    date_end = datetime.fromtimestamp(int(float(date_end))).strftime("%Y-%m-%d %H:%M:%S")
    cur.execute(f"""SELECT *     
        FROM {stock_symbol} 
        WHERE timestamp > '{date_begin}'
        AND timestamp < '{date_end}';""")
    stock = pd.DataFrame(cur.fetchall()).set_index(0)
    stock.rename(columns={0: 'Timestamp', 1: 'open', 2: 'High', 3:'Low',4:'Close',5:'Volume',6:'Trade Count',7:'VWAP'}, inplace=True)
    return stock

def get_stock_alpaca(stock_symbol, date_begin, date_end, timeframe = "1Min"):
    date_begin = datetime.fromtimestamp(int(float(date_begin))).strftime("%Y-%m-%d")
    date_end = datetime.fromtimestamp(int(float(date_end))).strftime("%Y-%m-%d")
    conn = create_connection('TradingBotDatabase.db')
    stock = alpaca.get_bars(stock_symbol, timeframe, date_begin, date_end).df   
    stock.to_sql(stock_symbol, conn, if_exists='append')
    print("stored in db")
    return stock

def get_stock_api(stock_symbol, date_begin, date_end):
    data = finnhub_client.stock_candles(stock_symbol, 'D', int(float(date_begin)), int(float(date_end)))
    df = pd.DataFrame(data)
    df['t'] = df['t'].apply(lambda x: datetime.datetime.fromtimestamp(x))
    stock = df[['t','o','h','l','c','v']].set_index('t')
    stock.rename(columns={'o': 'open', 'h': 'High', 'l':'Low','c':'Close','v':'Volume'}, inplace=True)
    return stock        

def print_trade_ideas():
    for i in trade_ideas:
        print(i.symbol, i.time_frame, i.position, i.time_posted, i.user)
        


In [123]:
trade_ideas = get_trade_ideas_db()

In [ ]:
users = calculate_total_profit(trade_ideas, 0.010, 0.005)

In [183]:
df = pd.DataFrame.from_dict(good_users_users, orient='index').sort_values(0, ascending=False)

In [194]:
good_users_users

{'MarketMoversReport': [-4.606330058558967, 18],
 'Johnny_Nickles': [-0.5429621874315131, 1],
 'lauralea': [-1.0284135310549374, 2],
 'holeyprofit': [0.4815242426750075, 2],
 'otwa2113': [-0.6612867039825971, 3]}

In [ ]:
good_users_users = calculate_total_profit_good_users(trade_ideas, 0.015, 0.015, good_users_list)

In [187]:
df = pd.DataFrame.from_dict(users, orient='index').sort_values(0, ascending=False)

In [188]:
df = df.rename(columns={0:"total profit", 1:"trade count"})

In [195]:
df[:10]

,total profit,trade count
Rocky88888888888,39.252155,14
myway999,30.979979,1
otwa2113,15.229575,27
lauralea,14.523260,19
Johnny_Nickles,13.937796,13
MarketMoversReport,12.410432,26
holeyprofit,11.602006,28
vmreyesa,10.667362,21
cdarbonne3344,9.735492,20
TheCryptoChartWhisperer,9.182223,10


In [216]:
good_users = df.loc[(df['total profit'] >= 10) & (df['trade count'] >= 10)]

In [217]:
good_users

,total profit,trade count
Rocky88888888888,39.252155,14
otwa2113,15.229575,27
lauralea,14.523260,19
Johnny_Nickles,13.937796,13
MarketMoversReport,12.410432,26
holeyprofit,11.602006,28
vmreyesa,10.667362,21


In [191]:
good_users_list = good_users.index.tolist()

In [192]:
good_users_list

['Rocky88888888888',
 'myway999',
 'otwa2113',
 'lauralea',
 'Johnny_Nickles',
 'MarketMoversReport',
 'holeyprofit',
 'vmreyesa']

In [139]:
conn = create_connection('TradingBotDatabase.db') 
df.to_sql("user_data", conn, if_exists='replace')


956

##### stock = get_stock_alpaca("TSLA", 1657344370.0, time.time() - 100000)

In [87]:
len(stock)

13878

In [ ]:
original_price = stock["Open"][0]

In [ ]:
original_price

In [ ]:
stock['% change'] = stock.apply(lambda x: 1 - (x['Open'] / original_price), axis = 1)

In [ ]:
stock

In [ ]:
print_trade_ideas()

In [118]:
import sqlite3
from sqlite3 import Error


def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by db_file
    :param db_file: database file
    :return: Connection object or None
    """
    conn = None
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)

    return conn

def create_table(conn, create_table_sql):
    """ create a table from the create_table_sql statement
    :param conn: Connection object
    :param create_table_sql: a CREATE TABLE statement
    :return:
    """
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def create_trade_idea(conn, idea):
    """
    Create a new project into the projects table
    :param conn:
    :param project:
    :return: project id
    """
    sql = ''' INSERT OR IGNORE INTO trade_ideas(symbol,time_frame,position,time_posted,user)
              VALUES(?,?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, idea)
    conn.commit()
    return cur.lastrowid

def create_new_table():
    
    database = r"TradingBotDatabase.db"


    # create a database connection
    conn = create_connection(database)
    sql_create_trade_ideas_table = """ CREATE TABLE IF NOT EXISTS stock_data (
                                        symbol text NOT NULL,
                                        timestamp timestamp NOT NULL,
                                        open float NOT NULL,
                                        high float NOT NULL,
                                        low float NOT NULL,
                                        close float NOT NULL,
                                        volumne integer NOT NULL,
                                        PRIMARY KEY(symbol)
                                    ); """
    # create tables
    if conn is not None:
        # create projects table
        create_table(conn, sql_create_trade_ideas_table)
    else:
        print("Error! cannot create the database connection.")

def add_trade_ideas(trade_ideas):
    database = r"TradingBotDatabase.db"


    # create a database connection
    conn = create_connection(database)

    # create tables
#     if conn is not None:
#         # create projects table
#         create_table(conn, sql_create_trade_ideas_table)
#     else:
#         print("Error! cannot create the database connection.")

    with conn:
        # tasks
        for i in trade_ideas:
            print(i)
            trade_idea = (i.symbol, i.time_frame, i.position, i.time_posted, i.user)
            create_trade_idea(conn, trade_idea)

        # create tasks
        
def add_stock_data(stock, symbol):
    database = r"TradingBotDatabase.db"


    # create a database connection
    conn = create_connection(database)

    with conn:
        # tasks
        for i in stock:
            print(i)
            trade_idea = (i.symbol, i.time_frame, i.position, i.time_posted, i.user)
            create_trade_idea(conn, trade_idea)

        # create tasks


In [ ]:
add_trade_ideas(trade_ideas)

In [120]:
dbideas= get_trade_ideas_db()

In [121]:
len(dbideas)

2917

In [ ]:
dbideas

In [ ]:
users

In [206]:
type(get_trade_ideas_db())

list